# 1.问题分析

## 1.1.问题类别

## 1.2.特征分析 

## 1.3.数据挖掘

# 2.建立基线

## 2.1.准备工具包

In [89]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math


from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import StratifiedKFold


%matplotlib inline

## 2.2.数据准备(不进行清洗)

In [2]:
train_raw = pd.read_csv('../data/public_raw.train.csv')

test_raw = pd.read_csv('../data/public_raw.test.csv')

train_raw['is_train']=1
test_raw['is_train']=0

df = pd.concat([train_raw, test_raw],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

## 2.3.特征工程(完全使用原始特征)

## 2.4.训练

In [108]:
# 准备训练集和测试集
train = df[df['is_train']==1]
test = df[df['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train','I_B','I_C'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train','I_B','I_C'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']


训练集数量:9000
测试集数量:8409


In [98]:
# 定义打分函数,  SCORE = 1/(1+RMSE)
def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))

# def cal_score(mse):
#     return np.divide(1, 1 + np.sqrt(mse))


# 定义交叉验证函数  
def cross_validate(models, X, Y, cv=5):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        #获取模型名
        name = str(i + 1) + '.' + str(model) 
#         print(i + 1,'- Model:', str(model).split('(')[0])
        print(name)
#         model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        model_name.append(name.split('(')[0])
        #计算metric
#         strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
#         folds=strat_k_fold.split(X,Y)
        nmse = cross_val_score(model, X, Y, cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        mse_avg.append(avg_mse)
        #计算分数
        scores = cal_score(-nmse)
        avg_score = np.average(scores)    
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

### 2.4.1.使用LightGBM模型

#### 2.4.1.1 使用LGBMRegressor和cross_val_score

In [99]:
lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 

In [100]:
# lgb2 = LGBMRegressor(n_estimators=800, max_depth=5, random_state=5, n_jobs=8, learning_rate=0.08, verbose_eval=500) 
# lgb3 = LGBMRegressor(n_estimators=700, max_depth=5, random_state=5, n_jobs=8) 
# lgb4 = LGBMRegressor(n_estimators=1000, max_depth=5, random_state=5, n_jobs=8) 
# lgb5 = LGBMRegressor(n_estimators=1100, max_depth=5, random_state=5, n_jobs=8) 

In [101]:
cross_validate(models=[lgb1], X=train_X, Y=train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=5, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=900,
       n_jobs=8, num_leaves=31, objective=None, random_state=5,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.41915703 0.03485974 0.02421466 0.02486614 0.08277516]
Score: [0.6070085  0.84266753 0.86534339 0.86378908 0.77657434]
Average MSE: 0.11717454717995457  - Score: 0.7910765690795137 



,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.117175,0.791077


#### 2.4.1.2 使用LGBMRegressor和自定义eval函数

In [73]:
ans=pd.DataFrame()
ans['ID']=test_X['ID']
ans.shape

(8409, 1)

In [103]:
# import lightgbm

from sklearn.model_selection import KFold



X = train_X.copy()
Y = train_Y.copy()
T = test_X.copy()

folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X, Y))

Y_predict = np.zeros(X.shape[0])
T_predict = []
# val_preds.append(np.zeros(train.shape[0]))
for n_fold, (train_idx, test_idx) in enumerate(folds):
    X_train, Y_train, X_test= X.iloc[train_idx], Y.iloc[train_idx], X.iloc[test_idx]
    lgb1.fit(X_train,Y_train)
    #预测每折中作为验证集的训练集
    Y_predict[test_idx] = lgb1.predict(X_test)
    #以该折训练出的模型预测测试集
    T_predict.append(lgb1.predict(T))
#     print('T_predict: \n'+str(T_predict))

print('Y_predict: ')
print(Y_predict)
#     T_pred.append(Y_predict)
# print('T_pred: ')
# print(T_pred)


print('local cv:',1/(1+np.sqrt(mean_squared_error(Y,Y_predict))))

Y_predict: 
[1.46104199 1.66531591 2.08794416 ... 9.6431778  9.43246941 0.39249984]
local cv: 0.8494442887294936


In [74]:
# local cv: 0.84944
y_predict_5 = np.mean(T_predict, axis=0)
print('y_predict_5: \n'+str(y_predict_5))

ans['y']=y_predict_5
ans['y'].describe()

y_predict_5: 
[0.39995031 1.32710263 2.18466183 ... 9.85877277 9.90556858 9.09118279]


count    8409.000000
mean        5.697065
std         3.460793
min        -0.182216
25%         2.504986
50%         5.691466
75%         8.887027
max        12.134582
Name: y, dtype: float64

In [76]:
# local cv: 0.84993
y_predict_10 = np.mean(T_predict, axis=0)
print('y_predict_10: \n'+str(y_predict_10))

ans['y_10']=y_predict_10
ans['y_10'].describe()

y_predict_10: 
[0.39687394 1.26673675 2.16500899 ... 9.83409458 9.94489728 9.07060563]


count    8409.000000
mean        5.696876
std         3.460119
min        -0.329737
25%         2.502208
50%         5.704800
75%         8.889595
max        12.244228
Name: y_10, dtype: float64

#### 2.4.1.3 使用自定义带有early stopping参数的LGBM模型

In [78]:
import lightgbm

from sklearn.model_selection import KFold

def my_val(preds, train_data):
    label = train_data.get_label()
    return 'score', 1/(1+np.sqrt(mean_squared_error(preds, label))), True
def my_obj(preds, train_data):
    labels = train_deata.get_label()

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.08,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 0
}
# 同样的条件下，此参数设置达到local cv: 0.8575832021441638


test_predicts = []
val_preds = []

for idx, seed in enumerate([1,2,3,4,5]):
    kf = KFold(5, shuffle=True, random_state=seed)
    
    val_preds.append(np.zeros(train.shape[0]))
    for n_fold, (tra_idx, val_idx) in enumerate(kf.split(train)):
        tra = train.iloc[tra_idx]
        val = train.iloc[val_idx]
        tst = test.copy()

        predictor = [c for c in tra.columns.tolist() if c not in['y','is_train','I_B','I_C']]

        train_set = lightgbm.Dataset(
            tra[predictor],
            tra['y']
        )

        validation_set = lightgbm.Dataset(
            val[predictor],
            val['y']
        )

        model = lightgbm.train(params, train_set, num_boost_round=5000,
                              valid_sets= [validation_set],
                              valid_names=['valid'],
                              early_stopping_rounds=100,
                               feval=my_val,
                              verbose_eval=500)

        val_preds[idx][val_idx] = model.predict(val[predictor])
        test_predicts.append(model.predict(tst[predictor]))

print('local cv:',1/(1+np.sqrt(mean_squared_error(train['y'],np.mean(val_preds,axis=0)))))

Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0576157	valid's score: 0.80643
Early stopping, best iteration is:
[686]	valid's l2: 0.0573527	valid's score: 0.806787
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0200062	valid's score: 0.876084
[1000]	valid's l2: 0.0179172	valid's score: 0.881947
[1500]	valid's l2: 0.0174487	valid's score: 0.883319
[2000]	valid's l2: 0.0173969	valid's score: 0.883472
Early stopping, best iteration is:
[1904]	valid's l2: 0.0173704	valid's score: 0.883551
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[345]	valid's l2: 0.0133098	valid's score: 0.896565
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0501147	valid's score: 0.817085
[1000]	valid's l2: 0.046937	valid's score: 0.821929
[1500]	valid's l2: 0.0455346	valid's score: 0.824138
[2000]	valid's l2: 0.0449449	valid's score: 0.825081
[2500]	valid's

In [79]:
# local cv: 0.857100
y_predict_earlystopping = np.mean(test_predicts, axis=0)
print('y_predict_earlystopping: \n'+str(y_predict_earlystopping))

ans['y_predict_earlystopping']=y_predict_earlystopping
ans['y_predict_earlystopping'].describe()

y_predict_earlystopping: 
[0.42516574 1.32970174 2.1882634  ... 9.90467328 9.87744583 9.13143841]


count    8409.000000
mean        5.696583
std         3.458596
min        -0.137579
25%         2.515775
50%         5.715023
75%         8.888176
max        12.085619
Name: y_predict_earlystopping, dtype: float64

### 2.4.2.使用LightGBM/XGBoost/RF/GBM多种模型训练后融合

In [128]:
#基学习器

xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=321, n_jobs=8)
xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=456, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=789, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=7, n_jobs=8)
forest2 = RandomForestRegressor(n_estimators=900, max_features='log2', random_state=9, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=900, max_features='sqrt', random_state=11, n_jobs=8) 

lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9, n_jobs=8)

regrs = [
    xgbt1, gbdt1, forest1, lgb1,
    xgbt2, gbdt2, forest2, lgb2,
    xgbt3, gbdt3, forest3, lgb3
]

In [129]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # Train_X: 原始训练集输入矩阵, Train_Y: 原始训练集输出矩阵, Test_X: 原始测试集输入矩阵
    def fit_predict(self, Train_X, Train_Y, Test_X):
        Train_X = np.array(Train_X)
        Train_Y = np.array(Train_Y)
        Test_X = np.array(Test_X)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(Train_X, Train_Y))       
        
        # 以基学习器预测结果为特征的 stacker训练数据 与 stacker预测数据
        # 原始训练集预测结果容器
        S_train = np.zeros((Train_X.shape[0], len(self.base_models)))
        # 原始测试集预测结果容器
        S_predict = np.zeros((Test_X.shape[0], len(self.base_models)))
        
        for n_model, regr in enumerate(self.base_models):
            print(n_model + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((Test_X.shape[0], self.n_splits))
            
            for n_fold, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train_fold, Y_train_fold, X_test_fold, Y_test_fold = Train_X[train_idx], Train_Y[train_idx], Train_X[test_idx], Train_Y[test_idx]
                print ('Fit fold', (n_fold+1), '...')
                regr.fit(X_train_fold, Y_train_fold)
                Y_pred = regr.predict(X_test_fold)
                # 每折训练得到的模型根据原始训练集中的测试折的输入矩阵预测
                S_train[test_idx, n_model] = Y_pred
                # 每折训练得到的模型根据原始测试集输入矩阵预测
                S_predict_i[:, n_fold] = regr.predict(Test_X)
            
            S_predict[:, n_model] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, Train_Y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, Train_Y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [130]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
5 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
6 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
7 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
8 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
9 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
10 Base model: GradientBoostingRegre

In [ ]:
#local CV: 0.7909689115097466

ans['y_n_shuffle'] = pred_stack
ans['y_n_shuffle'].describe()

In [115]:
#local CV: 0.8697213790417854

ans['y_shuffle'] = pred_stack
ans['y_shuffle'].describe()

count    8409.000000
mean        5.695812
std         3.459761
min        -0.131349
25%         2.505766
50%         5.704603
75%         8.887224
max        12.348426
Name: y_shuffle, dtype: float64